# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [64]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, concat, count, lit, udf, avg
from sklearn.metrics import classification_report

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
database_filepath = "mini_sparkify_event_data.json"

In [27]:
def load_data(database_filepath):
    '''
    The function creates a Spark session and loads the dataset from the json file into spark.
    '''
    # create a Spark session
    spark = SparkSession.builder \
        .master("local") \
        .appName("sparkify") \
        .getOrCreate()

    user_log = spark.read.json(database_filepath) # database_filepath = "mini_sparkify_event_data.json"
    return user_log
    pass
df = load_data('mini_sparkify_event_data.json')

In [28]:
def clean_data(user_log):
    '''
    The function drops rows ["userId", "sessionId"] have Missing Values or empty from the inputed dataset
                 add new cols ('Churn','chgrd','time_gap','sex') into dataset
                 flag 'cancelled user' as 1, 'others' as 0 in col 'Churn'
                 flag 'paid' usage in col 'chgrd'
                 flag 'gender' in numeric format in col 'chgrd'
                 get usage time by calculate max(ts) - min(ts) per user and saved the result in col 'time_gap'
    '''
    # drop rows ["userId", "sessionId"] have Missing Values or empty
    user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])
    user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

    # find whether users Cancellation Confirmation and then flag'cancelled user' as 1, 'others' as 0 in 'Churn'.
    flag_Cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
    user_log_valid = user_log_valid.withColumn("Churn", flag_Cancel_event("page"))
    windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
    user_log_valid = user_log_valid.withColumn("Churn", Fsum("Churn").over(windowval))

    # get numbers of page operations when the users' level = 'paid' and saved the numbers in column 'chgrd'.
    flag_changegrade_event = udf(lambda x: 1 if x == 'paid' else 0, IntegerType())
    user_log_valid = user_log_valid.withColumn("chgrd", flag_changegrade_event("level"))
    windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
    user_log_valid = user_log_valid.withColumn("chgrd", Fsum("chgrd").over(windowval))

    # identify users' gender, flag male as 1 and female as 0 in 'sex'
    flag_gender_event = udf(lambda x: 1 if x == "M" else 0, IntegerType())
    user_log_valid = user_log_valid.withColumn("sex", flag_gender_event("gender"))

    # create a temporary View and run SQL queries to get cleaned data df
    user_log_valid.createOrReplaceTempView("user_log_valid_table")
    # cleaned data includes user Id, genda, paid usage(chrgd), users' stay time (time_gap), and cancel or not flag (Churn)
    df = spark.sql(
        '''
        select t1.userId,sex,chgrd,time_gap,Churn
        from user_log_valid_table t1
        join
         (select userId, max(chgrd) as mchgrd,(max(ts) - min(ts))/1000 as time_gap
         from user_log_valid_table
         group by userId) t2
        on (t1.userId == t2.userId and t1.chgrd == t2.mchgrd)
        order by Churn
        '''
                      ).dropDuplicates()
    return df
    pass

df = clean_data(df)

In [60]:
#Not used in the main()
# Get YYYY-MM-DD-HH 
get_YYYY = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). year)
get_MM = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). month)
get_DD = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). day)
get_HH = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)

#user_log = user_log.withColumn("Hour", get_HH(user_log.ts))
#user_log = user_log.withColumn('Year',get_YYYY(user_log.ts))
#user_log = user_log.withColumn('Month',get_MM(user_log.ts))
#user_log = user_log.withColumn('Day',get_DD(user_log.ts))
#user_log.show()

# add 'state' column
splt = udf(lambda x:x.split(',')[1][1:3])
user_log_valid = user_log_valid.withColumn('ST',splt('location'))
#user_log_valid.select('ST').show()

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [30]:
def explore_data(df):
    '''
    The function displays the Sex distribution,Usage Time distribution,
    and paid usage Distribution between cancled users and noncancled users
    '''
    # display the sex distribution between cancled users and noncancled users
    print('Insights:\n 1 Sex Disparity of Cancled User \n')
    df.groupby(['Churn','sex']).count().show()

    # display the usage time distribution between cancled users and noncancled users
    print('\n 2 Usage Time Distribution \n')
    df.groupby('Churn').avg('time_gap').show()

    # display the paid usage (average of 'chgrd')distribution between cancled users and noncancled users
    print('\n 3 Usage paid usage Distribution \n')
    df.groupby('Churn').avg('chgrd').show()

    pass
explore_data(df)

Insights:
 1 Sex Disparity of Cancled User 

+-----+---+-----+
|Churn|sex|count|
+-----+---+-----+
|    0|  1|   89|
|    0|  0|   84|
|    1|  1|   32|
|    1|  0|   20|
+-----+---+-----+


 2 Usage Time Distribution 

+-----+-----------------+
|Churn|    avg(time_gap)|
+-----+-----------------+
|    0|4060011.456647399|
|    1|2031665.576923077|
+-----+-----------------+


 3 Usage paid usage Distribution 

+-----+------------------+
|Churn|        avg(chgrd)|
+-----+------------------+
|    0|1098.0173410404625|
|    1| 624.5384615384615|
+-----+------------------+



In [ ]:
# Not used
#user_log_valid.select("page").dropDuplicates().sort("page").show() #"auth",'level','method','page','sessionId','status'
#user_log_valid.select("userId").where(user_log_valid.auth == 'Cancelled').collect()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
def feature_engi(df):
    '''
    The function Combine the gender, usage time, and paid usage columns into a vector,
    and Scales the Vectors
    '''
    #Combine the gender, usage time, and paid usage columns into a vector
    assembler = VectorAssembler(inputCols=["sex","time_gap","chgrd"], outputCol="NumFeatures")
    df = assembler.transform(df)
    #Scale the Vectors
    scaler = StandardScaler(inputCol="NumFeatures", outputCol="features",withMean=True, withStd=False)
    scalerModel = scaler.fit(df)
    df = scalerModel.transform(df)
    return df
    pass

In [27]:
#Not used
#Normalize the Vectors
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures")
df = scaler.transform(df)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [31]:
def build_model():
    '''
    The function build model with
    machine pipeline which take in the combined and scaled vector column "ScaledNumFeatures" as input and output
    classification results on the 2 labels("Churn") in the dataset.
    '''
    indexer = StringIndexer(inputCol="Churn", outputCol="label")
    lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)
    pipeline = Pipeline(stages=[indexer,lr])
    #tune model
    paramGrid = ParamGridBuilder() \
        .addGrid(lr.regParam,[0.0, 0.1]) \
        .addGrid(lr.maxIter,[10, 20]) \
        .build()
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=MulticlassClassificationEvaluator(),
                              numFolds=3)
    return crossval
    pass
df = feature_engi(df)
model = build_model()

In [32]:
rest, validation = df.randomSplit([0.9, 0.1], seed=42)

In [38]:
cvmodel = model.fit(rest)

In [83]:
def evaluate_model(model, validation):
    '''
    The function reports the accuracy of the model prediction against validation dataset
    '''
    results = model.transform(validation)
    # using classification_report to get F1 score
    labels = np.unique(results.select ('prediction').collect())
    confusion_mat = classification_report(results.select ('label').collect(), results.select ('prediction').collect(),labels=labels)
    print(" Accuracy:\n")
    print((results.filter(results.label == results.prediction).count())/(results.count()))
    print(" Confusion Matrix:\n ", confusion_mat)
    return results
    pass
results = evaluate_model(cvmodel, validation)

 Accuracy:

0.5263157894736842
 Confusion Matrix:
               precision    recall  f1-score   support

        0.0       0.47      0.88      0.61         8
        1.0       0.75      0.27      0.40        11

avg / total       0.63      0.53      0.49        19



In [59]:
import pickle
def save_model(model, model_filepath):
    '''
    The function export tained model as a pickle file
    '''
    s = pickle.dumps(model.avgMetrics)
    with open(model_filepath,'wb+') as f: # mode is'wb+'，represents binary writen
        f.write(s)
    pass
save_model(cvmodel, 'cls.pkl')

In [80]:
#results_pd = results.toPandas()
labels = np.unique(result.select ('prediction').toPandas()collect())
confusion_mat = classification_report(result.select ('label').collect(), result.select ('prediction').collect(),labels=labels)

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [24]:
#Sparkify Project Workspace
# import libraries
import sys
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, concat, count, lit, udf, avg
from sklearn.metrics import classification_report

import datetime

import numpy as np
import pandas as pd
# % matplotlib inline
import matplotlib.pyplot as plt

def load_data(database_filepath):
    '''
    The function creates a Spark session and loads the dataset from the json file into spark.
    '''
    # create a Spark session
    spark = SparkSession.builder \
        .master("local") \
        .appName("sparkify") \
        .getOrCreate()

    user_log = spark.read.json(database_filepath) # database_filepath = "mini_sparkify_event_data.json"
    return user_log
    pass

def clean_data(user_log):
    '''
    The function drops rows ["userId", "sessionId"] have Missing Values or empty from the inputed dataset
                 add new cols ('Churn','chgrd','time_gap','sex') into dataset
                 flag 'cancelled user' as 1, 'others' as 0 in col 'Churn'
                 flag 'paid' usage in col 'chgrd'
                 flag 'gender' in numeric format in col 'chgrd'
                 get usage time by calculate max(ts) - min(ts) per user and saved the result in col 'time_gap'
    '''
    # drop rows ["userId", "sessionId"] have Missing Values or empty
    user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])
    user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

    # find whether users Cancellation Confirmation and then flag'cancelled user' as 1, 'others' as 0 in 'Churn'.
    flag_Cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
    user_log_valid = user_log_valid.withColumn("Churn", flag_Cancel_event("page"))
    windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
    user_log_valid = user_log_valid.withColumn("Churn", Fsum("Churn").over(windowval))

    # get numbers of page operations when the users' level = 'paid' and saved the numbers in column 'chgrd'.
    flag_changegrade_event = udf(lambda x: 1 if x == 'paid' else 0, IntegerType())
    user_log_valid = user_log_valid.withColumn("chgrd", flag_changegrade_event("level"))
    windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
    user_log_valid = user_log_valid.withColumn("chgrd", Fsum("chgrd").over(windowval))

    # identify users' gender, flag male as 1 and female as 0 in 'sex'
    flag_gender_event = udf(lambda x: 1 if x == "M" else 0, IntegerType())
    user_log_valid = user_log_valid.withColumn("sex", flag_gender_event("gender"))

    # create a temporary View and run SQL queries to get cleaned data df
    user_log_valid.createOrReplaceTempView("user_log_valid_table")
    # cleaned data includes user Id, genda, paid usage(chrgd), users' stay time (time_gap), and cancel or not flag (Churn)
    df = spark.sql(
        '''
        select t1.userId,sex,chgrd,time_gap,Churn
        from user_log_valid_table t1
        join
         (select userId, max(chgrd) as mchgrd,(max(ts) - min(ts))/1000 as time_gap
         from user_log_valid_table
         group by userId) t2
        on (t1.userId == t2.userId and t1.chgrd == t2.mchgrd)
        order by Churn
        '''
                      ).dropDuplicates()
    return df
    pass

def explore_data(df):
    '''
    The function displays the Sex distribution,Usage Time distribution,
    and paid usage Distribution between cancled users and noncancled users
    '''
    # display the sex distribution between cancled users and noncancled users
    print('Insights:\n 1 Sex Disparity of Cancled User \n')
    df.groupby(['Churn','sex']).count().show()

    # display the usage time distribution between cancled users and noncancled users
    print('\n 2 Usage Time Distribution \n')
    df.groupby('Churn').avg('time_gap').show()

    # display the paid usage (average of 'chgrd')distribution between cancled users and noncancled users
    print('\n 3 Usage paid usage Distribution \n')
    df.groupby('Churn').avg('chgrd').show()

    pass

def feature_engi(df):
    '''
    The function Combine the gender, usage time, and paid usage columns into a vector,
    and Scales the Vectors
    '''
    #Combine the gender, usage time, and paid usage columns into a vector
    assembler = VectorAssembler(inputCols=["sex","time_gap","chgrd"], outputCol="NumFeatures")
    df = assembler.transform(df)
    #Scale the Vectors
    scaler = StandardScaler(inputCol="NumFeatures", outputCol="features",withMean=True, withStd=False)
    scalerModel = scaler.fit(df)
    df = scalerModel.transform(df)
    return df
    pass

def build_model():
    '''
    The function build model with
    machine pipeline which take in the combined and scaled vector column "ScaledNumFeatures" as input and output
    classification results on the 2 labels("Churn") in the dataset.
    '''
    indexer = StringIndexer(inputCol="Churn", outputCol="label")
    lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)
    pipeline = Pipeline(stages=[indexer,lr])
    #tune model
    paramGrid = ParamGridBuilder() \
        .addGrid(lr.regParam,[0.0, 0.1]) \
        .addGrid(lr.maxIter,[10, 20]) \
        .build()
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=MulticlassClassificationEvaluator(),
                              numFolds=3)
    return crossval
    pass

def evaluate_model(model, validation):
     '''
    The function reports the accuracy of the model prediction against validation dataset
    '''
    results = model.transform(validation)
    # using classification_report to get F1 score
    labels = np.unique(results.select ('prediction').collect())
    confusion_mat = classification_report(results.select ('label').collect(), results.select ('prediction').collect(),labels=labels)
    print(" Accuracy:\n")
    print((results.filter(results.label == results.prediction).count())/(results.count()))
    print(" Confusion Matrix:\n ", confusion_mat)
    return results
    pass

def save_model(model, model_filepath):
    '''
    The function export tained model as a pickle file
    '''
    s = pickle.dumps(model)
    with open(model_filepath,'wb+') as f: # mode is'wb+'，represents binary writen
        f.write(s)
    pass

def main():
    '''
    The main() function combines and executes all the above modules.
    '''
    database_filepath, model_filepath = sys.argv[1:]
    print('Loading data...\n    DATABASE: {}'.format(database_filepath))
    user_log = load_data(database_filepath)

    # Train Test Split
    #break data set into 90% of training data and set aside 10%. Set random seed to 42.
    rest, validation = df.randomSplit([0.9, 0.1], seed=42)

    print('Building model...')
    model = build_model()

    #Train pipeline
    print('Training model...')
    model.fit(rest)

    print('Evaluating model...')
    evaluate_model(validation)

    print('Saving model...\n    MODEL: {}'.format(model_filepath))
    save_model(model, model_filepath)

    print('Trained model saved!')

if __name__ == '__main__':
    main()


Loading data...
    DATABASE: -f


AnalysisException: 'Path does not exist: file:/home/workspace/-f;'